In [1]:
import sympy as sp

In [5]:
x, L_fin, lambd, Ac, k, Bi, T_ext, T_0 = sp.symbols('x L_fin \lambda Ac k Bi T_ext T_0')

def fin_analytical(bc):

    # Define the temperature profile solution
    A = sp.Symbol('A')
    B = sp.Symbol('B')

    # Temperature function T(x)
    T = T_ext + A * sp.cosh(sp.sqrt(2 * Bi) * x/lambd) + B * sp.sinh(sp.sqrt(2 * Bi) * x/lambd)

    # Boundary condition at x = 0 (T(0) = T_0)
    boundary_condition_0 = sp.Eq(T.subs(x, 0), T_0)

    # Solve for A
    A_solution = sp.solve(boundary_condition_0, A)[0]

    # Substitute the value of A in T(x)
    T = T.subs(A, A_solution)

    dTdx = sp.diff(T, x)

    if bc == 'free_end':
        boundary_condition_L_fin = sp.Eq( dTdx.subs(x, L_fin), -Bi/lambd * (T.subs(x, L_fin) - T_ext) )
        B_solution = sp.solve(boundary_condition_L_fin, B)[0]
    elif bc == 'adiabatic':
        boundary_condition_L_fin = sp.Eq( dTdx.subs(x, L_fin), 0)
        B_solution = sp.solve(boundary_condition_L_fin, B)[0]
    else:
        raise ValueError('Boundary condition not recognized')

    T = T.subs(B, B_solution)
    dTdx = dTdx.subs(B, B_solution)

    expression = (T-T_ext) / (T_0-T_ext)
    expression = sp.simplify(expression)

    if bc == "adiabatic":
        expression = sp.simplify(expression * sp.cosh(sp.sqrt(2 * Bi) * L_fin/lambd)) / sp.cosh(sp.sqrt(2 * Bi) * L_fin/lambd)

    gamma = sp.diff(expression,x).subs(x, 0)
    gamma = - k * Ac * sp.simplify(gamma)

    Qdot =  gamma * (T_0 - T_ext)

    return T, dTdx, expression, gamma, Qdot

gamma_free_end = fin_analytical('free_end')[3]
gamma_adia = fin_analytical('adiabatic')[3]

In [6]:
print(gamma_adia)

sqrt(2)*Ac*sqrt(Bi)*k*tanh(sqrt(2)*sqrt(Bi)*L_fin/\lambda)/\lambda


In [7]:
T_free_end = fin_analytical('free_end')[0]
T_adia = fin_analytical('adiabatic')[0]

In [8]:
print(T_free_end)

T_ext + (T_0 - T_ext)*cosh(sqrt(2)*sqrt(Bi)*x/\lambda) - (sqrt(2)*sqrt(Bi)*T_0*sinh(sqrt(2)*sqrt(Bi)*L_fin/\lambda) - sqrt(2)*sqrt(Bi)*T_ext*sinh(sqrt(2)*sqrt(Bi)*L_fin/\lambda) + Bi*T_0*cosh(sqrt(2)*sqrt(Bi)*L_fin/\lambda) - Bi*T_ext*cosh(sqrt(2)*sqrt(Bi)*L_fin/\lambda))*sinh(sqrt(2)*sqrt(Bi)*x/\lambda)/(sqrt(2)*sqrt(Bi)*cosh(sqrt(2)*sqrt(Bi)*L_fin/\lambda) + Bi*sinh(sqrt(2)*sqrt(Bi)*L_fin/\lambda))


In [18]:
print(sp.integrate(T_adia, (x, 0, L_fin)))

L_fin*T_ext + Piecewise((sqrt(2)*\lambda*(T_0 - T_ext)*sinh(sqrt(2)*sqrt(Bi)*L_fin/\lambda)/(2*sqrt(Bi)) - sqrt(2)*\lambda*(T_0 - T_ext)*cosh(sqrt(2)*sqrt(Bi)*L_fin/\lambda)*tanh(sqrt(2)*sqrt(Bi)*L_fin/\lambda)/(2*sqrt(Bi)) + sqrt(2)*\lambda*(T_0 - T_ext)*tanh(sqrt(2)*sqrt(Bi)*L_fin/\lambda)/(2*sqrt(Bi)), (Bi > -oo) & (Bi < oo) & Ne(Bi, 0)), (L_fin*(T_0 - T_ext), True))
